<a href="https://colab.research.google.com/github/quicksilverTrx/mining_of_massive_datasets/blob/main/colab_notebooks/Frequent_Pattern_Mining_in_Spark_Colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 63kB/s 
     |████████████████████████████████| 204kB 38.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=4a42c27eb447a7e54a2e556294b6bc02be044fcf9504002d55feb0263dc249b3
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [7]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [8]:
'''
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
    '''

'\n!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip\n!unzip ngrok-stable-linux-amd64.zip\nget_ipython().system_raw(\'./ngrok http 4050 &\')\n!curl -s http://localhost:4040/api/tunnels | python3 -c     "import sys, json; print(json.load(sys.stdin)[\'tunnels\'][0][\'public_url\'])"\n    '

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [28]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [29]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [30]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [32]:
# YOUR CODE HERE
product_name=orders.join(products, on='product_id')
group_order=orders.join(products, on='product_id').groupBy("order_id").agg(collect_list('product_name').alias("product_name"))

In [38]:
group_order.limit(5).collect()

[Row(order_id=1342, product_name=['Bag of Organic Bananas', 'Seedless Cucumbers', 'Organic Mandarins', 'Organic Strawberries', 'Versatile Stain Remover', 'Pink Lady Apples', 'Chicken Apple Sausage', 'Raw Shrimp']),
 Row(order_id=1591, product_name=['Cracked Wheat', 'Organic Greek Whole Milk Blended Vanilla Bean Yogurt', 'Navel Oranges', 'Spinach', 'Original Patties (100965) 12 Oz Breakfast', 'Cinnamon Multigrain Cereal', 'Chewy 25% Low Sugar Chocolate Chip Granola', 'Uncured Genoa Salami', 'Natural Vanilla Ice Cream', 'Lemon Yogurt', 'Strawberry Rhubarb Yoghurt', 'Garlic', 'Pure Vanilla Extract', 'Lower Sugar Instant Oatmeal  Variety', 'Organic Bunny Fruit Snacks Berry Patch', 'Buttermilk Waffles', 'Granny Smith Apples', 'Medium Scarlet Raspberries', 'Banana', 'Strawberry Banana Smoothie', 'Green Machine Juice Smoothie', 'Ultra Thin Sliced Provolone Cheese', 'Oven Roasted Turkey Breast', 'Original Turkey Burgers Smoke Flavor Added', 'Original Whole Grain Chips', 'Goldfish Pretzel Baked

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [19]:
# YOUR CODE HERE
group_order.printSchema()
group_order.limit(15).show()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: array (nullable = false)
 |    |-- element: integer (containsNull = false)

+--------+--------------------+
|order_id|          product_id|
+--------+--------------------+
|    1342|[13176, 30827, 14...|
|    1591|[17203, 44008, 48...|
|    4519|             [29270]|
|    4935|             [45190]|
|    6357|[37524, 33731, 43...|
|   10362|[22451, 46823, 47...|
|   19204|[25783, 26165, 45...|
|   29601|[4472, 44329, 271...|
|   31035|[13176, 8174, 407...|
|   40011|[27292, 7905, 219...|
|   46266|[23029, 38558, 34...|
|   51607|[41390, 42752, 14...|
|   58797|[13176, 3265, 983...|
|   61793|[43352, 8859, 196...|
|   67089|[24852, 47766, 27...|
+--------+--------------------+



In [40]:
from pyspark.ml.fpm import FPGrowth
fpgrowth=FPGrowth(itemsCol="product_name",minSupport=0.01 , minConfidence=0.05)
model= fpgrowth.fit(group_order)
# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(group_order).show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Strawber...| 3074|
|[Organic Strawber...| 2174|
|[Organic Baby Spi...| 9784|
|[Organic Baby Spi...| 1639|
|[Organic Baby Spi...| 2236|
|[Organic Baby Spi...| 2000|
|       [Large Lemon]| 8135|
|[Large Lemon, Ban...| 2158|
|   [Organic Avocado]| 7409|
|[Organic Avocado,...| 1349|
|[Organic Avocado,...| 1402|
|[Organic Avocado,...| 2216|
|[Organic Hass Avo...| 7293|
|[Organic Hass Avo...| 1539|
|[Organic Hass Avo...| 2420|
|      [Strawberries]| 6494|
|[Strawberries, Ba...| 1948|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+-------

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [46]:
model.freqItemsets.count()


120

In [45]:
model.associationRules.count()

32

Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [44]:
# YOUR CODE HERE
fpgrowth1=FPGrowth(itemsCol="product_name",minSupport=0.001 , minConfidence=0.05)
model1= fpgrowth1.fit(group_order)
# Display frequent itemsets.
model1.freqItemsets.show()

# Display generated association rules.
model1.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model1.transform(group_order).show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|[Organic Tomato B...|  772|
|[Organic Tomato B...|  175|
|[Organic Tomato B...|  144|
|[Organic Tomato B...|  179|
|[Organic Spinach ...|  475|
|[Whole Milk Ricot...|  347|
| [Medium Salsa Roja]|  275|
|    [Ground Buffalo]|  231|
|       [Tonic Water]|  194|
|[Original Coconut...|  173|
|[Low-Fat Strawber...|  152|
|[Organic SprouTof...|  137|
|            [Banana]|18726|
|[Fruit Punch Spor...|  275|
|[Kitchen Cheese E...|  230|
|[Country White Br...|  194|
|[Soft & Smooth Wh...|  173|
|[Natural Liquid L...|  152|
|[Bag of Organic B...|15480|
|[Organic Large Gr...|  769|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+-------

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!